### ![](../../img/installation-ico.png) Install

In [ ]:
pip install langchain langchain-community langchain-google-vertexai google.auth

Fonctionnalités nécessaires à l'exploitation de fichiers PDF

In [ ]:
pip install pypdf

FAISS pour LangChain

In [ ]:
pip install faiss-cpu

### ![](../../img/package-ico.png) Imports

In [ ]:
import os
import google.auth

from langchain_google_vertexai import VertexAI

# pour la récupération de données depuis un fichier PDF
from langchain_community.document_loaders import PyPDFLoader

# pour redécouper les documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

# FAISS Vectore Store
from langchain_community.vectorstores import FAISS


# modèle d'embedding de documents
from langchain_google_vertexai import VertexAIEmbeddings

# récupération du prompt et construction de la châine
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate

# permet de passer un paramètre entre les différents composants d'une chaîne
from langchain_core.runnables import RunnablePassthrough

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

### ![](../../img/parametrage-ico.png) Paramétrages


In [ ]:
# Gestion credentials VertexAI

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../settings/credentials.json"
google.auth.default()

In [ ]:
# Étape 1 : Charger le document PDF avec PyPDFLoader
pdf_path = "./TechWave.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print("Number of extracted documents : ", len(documents))

In [ ]:
# Étape 2 : Diviser le document en morceaux plus petits
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

print("Number of splitted documents : ", len(split_documents))

In [ ]:
# Étape 3 : Stocker les documents divisés dans un vector store FAISS
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko-multilingual@001")
vectorstore = FAISS.from_documents(split_documents, embeddings)

In [ ]:
# Étape 4 : Définir le LLM
llm = VertexAI()

In [ ]:
# Étape 5 : Définir le ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [        
        HumanMessagePromptTemplate.from_template("""You are an assistant for question-answering tasks.
        Use the following pieces of retrieved context to answer the question.
        Use three sentences maximum and keep the answer concise.
        Question: {input} 
        Context: {context} 
        Answer:
    """),
    ]
)

In [ ]:
# Étape 6 : Utiliser FAISS comme retriever
retriever = vectorstore.as_retriever()

# Étape 7 : Chaîner les éléments
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

### ![](../../img/jouer-ico.png) Exécution

In [ ]:
# Exemple d'utilisation
question = "Quelle entreprise a rédigé ce document ?"
response = retrieval_chain.invoke({"input": question})

print(response["answer"])

## Aller plus loin

In [ ]:
# Définir le modèle de réponse avec Pydantic
class Response(BaseModel):
    answer: str = Field(description="Answer to the question")


# Définir le JsonOutputParser
parser = JsonOutputParser(pydantic_object=Response)

# Définir le ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        # Message système pour spécifier le comportement
        SystemMessage(content=("You are a helpful assistant.")),
        # Message système précisant le format de sortie
        SystemMessage(content=(parser.get_format_instructions())),
        # Message humain à partir d'un template
        HumanMessagePromptTemplate.from_template(
            """Answer the question based only on the following context:
        <context>
        {context}
        <context>

        Question: {question}
        """
        ),
    ]
)

manual_retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | parser
)

question = "Quelle entreprise a rédigé ce document ?"
manual_response = manual_retrieval_chain.invoke(question)
print(manual_response)